In [24]:
!pip -q install PyPDF2 crewai google-generativeai gradio
!npm install -g netlify-cli


⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼
changed 1106 packages in 28s
⠴
⠴226 packages are looking for funding
⠴  run `npm fund` for details
⠴

In [25]:
import os
import google.generativeai as genai
from google.colab import userdata
from google.colab import files
# Set your API key here
os.environ["GOOGLE_API_KEY"] = userdata.get("gemini_api2")

# genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
gemini_api_key=userdata.get("gemini_api2")

In [26]:
from crewai import LLM
# llm = LLM(model="gemini/gemini-1.5-flash", api_key=gemini_api_key)
llm = LLM(model="gemini/gemini-2.5-flash", api_key=gemini_api_key)

In [57]:
import os
import re
import uuid
import json
import shutil
import zipfile
import subprocess
from PyPDF2 import PdfReader
from crewai import Agent, Task, Crew, LLM
from google.colab import files

# # Set your LLM
# llm = LLM(model="gpt-4", api_key="YOUR_OPENAI_API_KEY")

# ------------ SUBFUNCTIONS ------------

def upload_files():
    print("📄 Upload your resume PDF:")
    pdf_uploaded = files.upload()

    print("🖼️ Upload your profile photo (JPG/PNG):")
    img_uploaded = files.upload()

    pdf_path = list(pdf_uploaded.keys())[0]
    photo_path = list(img_uploaded.keys())[0]
    return pdf_path, photo_path

def extract_text_from_pdf(pdf_file):
    reader = PdfReader(pdf_file)
    return "\n".join(page.extract_text() for page in reader.pages if page.extract_text())

def create_agents(profile_text, photo_filename):
    extractor = Agent(
        role="Resume Extractor",
        goal="Extract structured info from resumes",
        backstory="Expert at understanding resume structure.",
        llm=llm,
        verbose=False
    )

    builder = Agent(
        role="Portfolio Website Generator",
        goal="Generate HTML/CSS personal portfolios with user image",
        backstory="Frontend developer with UX/UI focus.",
        llm=llm,
        verbose=False
    )

    extract_task = Task(
        description=f"Extract name, summary, experience, skills, education from:\n\n{profile_text}",
        expected_output="Structured data with name, title, experience, education, skills",
        agent=extractor
    )

    builder_task = Task(
        description=f"""Use the structured info to generate a responsive HTML+CSS portfolio.

        - Image: {photo_filename}
        - Use <img src="{photo_filename}" alt="Profile photo">
        - Output two code blocks: ```html ... ``` and ```css ... ```
        """,
        expected_output="Two code blocks: one for index.html and one for style.css.",
        agent=builder,
        depends_on=[extract_task]
    )

    return Crew(agents=[extractor, builder], tasks=[extract_task, builder_task], verbose=False)

def parse_html_css(raw_output):
    html_match = re.search(r"```html(.*?)```", raw_output, re.DOTALL)
    css_match = re.search(r"```css(.*?)```", raw_output, re.DOTALL)
    html = html_match.group(1).strip() if html_match else "<!-- No HTML -->"
    css = css_match.group(1).strip() if css_match else "/* No CSS */"
    return html, css

def write_portfolio_files(html, css, photo_filename, folder):
    os.makedirs(folder, exist_ok=True)
    with open(os.path.join(folder, "index.html"), "w") as f: f.write(html)
    with open(os.path.join(folder, "style.css"), "w") as f: f.write(css)
    shutil.copy(photo_filename, os.path.join(folder, photo_filename))

def deploy_to_netlify(folder, site_id):
    !netlify login

    !netlify init --manual --name portfolio-{site_id} --path {folder} --json > deploy_log.json
    !netlify deploy --dir={folder} --prod --json > deploy_result.json

    with open("deploy_result.json") as f:
        return json.load(f).get("url")



In [55]:
from google.colab import userdata

vercel_token = userdata.get('vercel_token')

In [19]:
import os, shutil, zipfile

def convert_portfolio_to_vite(portfolio_dir="portfolio", output_dir="vite_portfolio"):
    os.makedirs(output_dir, exist_ok=True)

    # 1. Copy existing files
    for file in os.listdir(portfolio_dir):
        shutil.copy(os.path.join(portfolio_dir, file), output_dir)

    # 2. Create vite.config.js
    with open(os.path.join(output_dir, "vite.config.js"), "w") as f:
        f.write("""
export default {
  root: '.',
  build: {
    outDir: 'dist',
    rollupOptions: {
      input: 'index.html'
    }
  }
}
""")

    # 3. Create package.json
    with open(os.path.join(output_dir, "package.json"), "w") as f:
        f.write("""
{
  "name": "vite-portfolio",
  "version": "1.0.0",
  "scripts": {
    "dev": "vite",
    "build": "vite build",
    "preview": "vite preview"
  },
  "devDependencies": {
    "vite": "^5.0.0"
  }
}
""")

    # 4. Zip for upload
    zip_path = f"{output_dir}.zip"
    with zipfile.ZipFile(zip_path, 'w') as zipf:
        for root, _, files in os.walk(output_dir):
            for file in files:
                full_path = os.path.join(root, file)
                arcname = os.path.relpath(full_path, output_dir)
                zipf.write(full_path, arcname)

    print(f"✅ Vite project ready at: {output_dir}")
    print(f"📦 Zipped to: {zip_path}")
    return zip_path


In [48]:
import os
import json
import base64
import requests

def deploy_static_site(folder_path, vercel_token, project_name="portfolio-agent"):
    files = []

    for root, _, filenames in os.walk(folder_path):
        for filename in filenames:
            full_path = os.path.join(root, filename)
            rel_path = os.path.relpath(full_path, folder_path).replace("\\", "/")
            with open(full_path, "rb") as f:
                encoded = base64.b64encode(f.read()).decode()
            files.append({
                "file": rel_path,
                "data": encoded,
                "encoding": "base64"
            })

    payload = {
        "name": project_name,
        "target": "production",
        "files": files,
        "projectSettings": {
            "framework": "vite"  # use "vite" to avoid framework errors
        },
        # Leave both unset for static-only sites with no build
        "buildCommand": None,
        "outputDirectory": None
    }

    headers = {
        "Authorization": f"Bearer {vercel_token}",
        "Content-Type": "application/json"
    }

    print("🚀 Deploying to Vercel...")

    response = requests.post(
        "https://api.vercel.com/v13/deployments",
        headers=headers,
        data=json.dumps(payload)
    )

    if response.status_code == 200:
        url = response.json().get("url")
        public_url = f"https://{url}"
        public_url=extract_public_vercel_url(public_url)
        print(f"✅ Site deployed successfully: {public_url}")
        return public_url
    else:
        print(f"❌ Deployment failed with status {response.status_code}")
        print(response.text)
        return None


In [50]:
from urllib.parse import urlparse

def extract_public_vercel_url(deployment_url):
    parsed = urlparse(deployment_url)
    hostname = parsed.hostname or ""
    # Remove everything after the first 2 hyphen-separated parts
    parts = hostname.split(".")[0].split("-")
    if len(parts) >= 2:
        public_subdomain = "-".join(parts[:2])
        return f"https://{public_subdomain}.vercel.app"
    else:
        return deployment_url  # fallback


In [23]:
import uuid

def generate_portfolio_and_deploy(vercel_token):
    # Step 1: Upload and extract
    pdf_path, photo_path = upload_files()
    profile_text = extract_text_from_pdf(pdf_path)

    # Step 2: Run CrewAI agents
    crew = create_agents(profile_text, photo_path)
    result = crew.kickoff()

    # Step 3: Parse generated HTML/CSS
    html, css = parse_html_css(result.tasks_output[-1].raw)

    # Step 4: Create unique portfolio folder
    site_id = str(uuid.uuid4())[:8]
    raw_folder = f"portfolio_{site_id}"
    os.makedirs(raw_folder, exist_ok=True)

    # Write HTML/CSS files
    with open(os.path.join(raw_folder, "index.html"), "w") as f:
        f.write(html)
    with open(os.path.join(raw_folder, "style.css"), "w") as f:
        f.write(css)

    # Copy image to folder
    image_dest = os.path.join(raw_folder, os.path.basename(photo_path))
    shutil.copy(photo_path, image_dest)

    # Step 5: Convert to Vite-compatible structure
    vite_folder = f"vite_{site_id}"
    zip_path = convert_portfolio_to_vite(portfolio_dir=raw_folder, output_dir=vite_folder)

    # Step 6: Deploy to Vercel
    live_url = deploy_static_site(vite_folder, vercel_token, project_name=f"portfolio-{site_id}")

    return live_url


In [58]:
 generate_portfolio_and_deploy(vercel_token)

📄 Upload your resume PDF:

Saving indira_resume_.pdf to indira_resume_ (9).pdf

🖼️ Upload your profile photo (JPG/PNG):

Saving WhatsApp Image 2025-07-16 at 17.48.11.jpeg to WhatsApp Image 2025-07-16 at 17.48.11 (6).jpeg

✅ Vite project ready at: vite_00cd0b6e

📦 Zipped to: vite_00cd0b6e.zip

🚀 Deploying to Vercel...

✅ Site deployed successfully: https://portfolio-00cd0b6e.vercel.app

'https://portfolio-00cd0b6e.vercel.app'